# Introduction

People’s life expectancy is increasing throughout the world as a result of improved living standards and medical advances. However, the natural ageing process is accompanied by physiological changes which can have significant consequences for mobility. Some common changes include: Reduced flexibility and strength; impairment of visual perception; increased vulnerability to bone fracture; etc. [[1]](https://link.springer.com/article/10.1007/s12062-016-9168-9) 

In order to create a transportation service that is conducive for older poeple, we addressed:


*   Accessibilty
*   Affordability
*   Comfort



We addressed them by analyzing Senior Citizen's mobility patterns, optimizing the transportation system, and proposing a model for an integrated transportation service.

## The Approach

We covered the following:



*   **Data Exploration:** We explored the datasets tto understand the mobility patterns of Senior Citizens.
*   **Route Optimization:** For each unique route ID, we created an optimal transportation route to ease **accessibility** and also save cost. By saving cost through route optimization, we can provide an **affordable** transportation system for Senior Citizens.
*    **Integrated Transportation Service:** We provided alternative transportation service and also structured transportation capacity to provide **comfort** for Senior Citizens.



# Development

Let's get started by importing necessary libraries and our Senior Citizen's datasets.

In [81]:
#importing libraries

import pandas as pd
import csv
import numpy as np

In [21]:
#importing the bus_routes dataset

bus_routes= pd.read_csv("/content/BusRoutes.csv")
bus_routes

,Unnamed: 0,IDRoute,linkid
0,0,1,537277300
1,1,1,537277673
2,2,1,537277754
3,3,1,537277777
4,4,1,537277795
...,...,...,...
48307,48307,166,1226896489
48308,48308,166,1226896507
48309,48309,166,1226896508
48310,48310,166,1226896568


From the above, we have an unnamed column. We don't need that, so we'll remove it

In [22]:
#removing unnamed columns

bus_routes = bus_routes.set_index('IDRoute')
bus_routes

,linkid
IDRoute,
1,537277300
1,537277673
1,537277754
1,537277777
1,537277795
...,...
166,1226896489
166,1226896507
166,1226896508


In [29]:
#importing senior monility dataset

senior_mobility = pd.read_csv("/content/Senior_TIM_v1.csv")
senior_mobility

,Unnamed: 0,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
0,0,80216810,R1 - AM Lisboa,Lisboa,Lisboa,2013.681187
1,1,80216810,R1 - AM Lisboa,Setúbal,Palmela,7688.296396
2,2,80216810,R1 - AM Lisboa,Lisboa,Oeiras,220.647248
3,3,80216810,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,233.203075
4,4,80216812,R1 - AM Lisboa,Lisboa,Lisboa,261.606014
...,...,...,...,...,...,...
121048,121048,1223824210,R1 - AM Lisboa,Setúbal,Barreiro,1877.243984
121049,121049,1223824210,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121050,121050,1223824211,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121051,121051,1223824211,R1 - AM Lisboa,Setúbal,Barreiro,961.962684


Similarly, we'll remove the unnamed column

In [30]:
#removing the unnamed column

senior_mobility = senior_mobility.drop("Unnamed: 0",axis=1)
#senior_mobility = senior_mobility.set_index('linkid')
senior_mobility

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
0,80216810,R1 - AM Lisboa,Lisboa,Lisboa,2013.681187
1,80216810,R1 - AM Lisboa,Setúbal,Palmela,7688.296396
2,80216810,R1 - AM Lisboa,Lisboa,Oeiras,220.647248
3,80216810,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,233.203075
4,80216812,R1 - AM Lisboa,Lisboa,Lisboa,261.606014
...,...,...,...,...,...
121048,1223824210,R1 - AM Lisboa,Setúbal,Barreiro,1877.243984
121049,1223824210,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121050,1223824211,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121051,1223824211,R1 - AM Lisboa,Setúbal,Barreiro,961.962684


Let's explore the mobility dataset

In [25]:
senior_mobility['Region_of_Origin'].unique()

array(['R1 - AM Lisboa', 'R2 - AM Porto'], dtype=object)

In [26]:
senior_mobility['County_of_Origin'].unique()

array(['Lisboa', 'Palmela', 'Oeiras', 'Vila Franca de Xira', 'Sintra',
       'Barreiro', 'Seixal', 'Almada', 'Amadora', 'Mafra', 'Odivelas',
       'Sesimbra', 'Maia', 'Loures', 'Porto', 'Gondomar', 'Setúbal',
       'Cascais', 'Montijo', 'Santa Maria da Feira', 'Matosinhos',
       'Moita', 'Alcochete'], dtype=object)

In [27]:
senior_mobility['District_of_Origin'].unique()

array(['Lisboa', 'Setúbal', 'Porto', 'Aveiro'], dtype=object)

In [32]:
len(senior_mobility['linkid'].unique())

75041

There are 2 regions in our dataset, 23 counties, and 4 districts. 

For ease, we'll be streamlining our model to a particular region. This solution can be extended across regions.

In [38]:
r1 = senior_mobility.loc[senior_mobility['Region_of_Origin'] == "R1 - AM Lisboa"]
r2 = senior_mobility.loc[senior_mobility['Region_of_Origin'] == "R2 - AM Porto"]
all_id = []
for i_d in r1["linkid"]:
    for i_d2 in r2["linkid"]:
        if (i_d == i_d2) and (i_d not in all_id):
            #print(senior_mobility.loc[senior_mobility['linkid'] == i_d])
            all_id.append(i_d)


KeyError: ignored

There are 2 different regions in the dataset. Looking at the first 4 rows in the senior_mobility dataset, we notice that the routes cut across different regions,

That is, the transportation consists of both inter-region and intra-region mobility. 

Let's consider intra-region mobility by looking at the "R1 - AM Lisboa" Region.

In [72]:
all_id = [str(id_) for id_ in all_id]
senior_mobility['linkid'] = [int(id__) for id__ in senior_mobility['linkid']]

In [76]:
#those moving between Lisboa and Porto regions

inter_region = senior_mobility.loc[senior_mobility['linkid'].isin(all_id)]
inter_region

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
41,80231987,R1 - AM Lisboa,Setúbal,Sesimbra,545.930444
42,80231987,R1 - AM Lisboa,Lisboa,Lisboa,72.235706
43,80231987,R2 - AM Porto,Porto,Maia,664.650224
44,80231987,R1 - AM Lisboa,Lisboa,Loures,231.154260
45,80231987,R1 - AM Lisboa,Setúbal,Barreiro,272.070254
...,...,...,...,...,...
120997,1223012250,R1 - AM Lisboa,Lisboa,Lisboa,108.828102
120998,1223012250,R2 - AM Porto,Porto,Gondomar,1495.463004
120999,1223012250,R2 - AM Porto,Porto,Porto,4630.210015
121000,1223012251,R2 - AM Porto,Porto,Porto,303.620329


In [78]:
#those moving inside Lisboa

intra_lisbon = r1.loc[~r1['linkid'].isin(all_id)]
intra_lisbon 

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
0,80216810,R1 - AM Lisboa,Lisboa,Lisboa,2013.681187
1,80216810,R1 - AM Lisboa,Setúbal,Palmela,7688.296396
2,80216810,R1 - AM Lisboa,Lisboa,Oeiras,220.647248
3,80216810,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,233.203075
4,80216812,R1 - AM Lisboa,Lisboa,Lisboa,261.606014
...,...,...,...,...,...
121048,1223824210,R1 - AM Lisboa,Setúbal,Barreiro,1877.243984
121049,1223824210,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121050,1223824211,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121051,1223824211,R1 - AM Lisboa,Setúbal,Barreiro,961.962684


## Optimizing Routes

In [84]:
intra_lisbon['Average_Daily_SeniorPopulation_Travelling'] = intra_lisbon['Average_Daily_SeniorPopulation_Travelling'].apply(np.ceil)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
intra_lisbon['Average_Daily_SeniorPopulation_Travelling'] = [int(id__) for id__ in intra_lisbon['Average_Daily_SeniorPopulation_Travelling']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
intra_lisbon

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
0,80216810,R1 - AM Lisboa,Lisboa,Lisboa,2014
1,80216810,R1 - AM Lisboa,Setúbal,Palmela,7689
2,80216810,R1 - AM Lisboa,Lisboa,Oeiras,221
3,80216810,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,234
4,80216812,R1 - AM Lisboa,Lisboa,Lisboa,262
...,...,...,...,...,...
121048,1223824210,R1 - AM Lisboa,Setúbal,Barreiro,1878
121049,1223824210,R1 - AM Lisboa,Setúbal,Palmela,506
121050,1223824211,R1 - AM Lisboa,Setúbal,Palmela,506
121051,1223824211,R1 - AM Lisboa,Setúbal,Barreiro,962


In [90]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

routes = intra_lisbon['linkid']
y_pos = np.arange(len(routes))
travelers = intra_lisbon['Average_Daily_SeniorPopulation_Travelling']

plt.bar(y_pos, travelers, align='center', alpha=0.5)
plt.xticks(y_pos, routes)
plt.ylabel('Routes')
plt.title('Average_Daily_SeniorPopulation_Travelling')

plt.show()

KeyboardInterrupt: ignored

To create an optimal transportation system, the buses will stop at different counties in each route, carrying available passengers as some are dropping.

To make this stops optimal, we need to get the required amount of buses that should be running at a given time. 

Following the assumption that Senior Citizens only move 12 hours in a day (7am to 7pm) because they are fragile an might have issues moving at night.

We evaluate the number of buses required on each route.

In [100]:
mobile_routes = intra_lisbon.loc[intra_lisbon['linkid'] == 80216810]

avg = mobile_routes["Average_Daily_SeniorPopulation_Travelling"]

for num in avg:
  #this shows the avg number of travelers in every hour
  avg = num/12
  #print (num/12)

  #given that a bus has 50 people capacity, then the avg is divided by 50
  cap = avg/50
  print(cap)

3.356666666666667
12.815
0.36833333333333335
0.39


Hence, we need approximately 13 buses every hour at a time when moving across this route. Given that a particular number of people will stop at each locat

In [93]:
intra_lisbon['linkid']

0           80216810
1           80216810
2           80216810
3           80216810
4           80216812
             ...    
121048    1223824210
121049    1223824210
121050    1223824211
121051    1223824211
121052    1223824212
Name: linkid, Length: 97882, dtype: int64

In [ ]:
def buses(travelers, stops):
  


# Future Work

To create a better model, we need to consider time when we have more senior citizens on the road. Is it 8am, 2pm, or *pm? That will help us better optimize the number of buses on the road
